### feature합치기
#### investors, historical(매일 거래량 등) and 분류항목 합치기

In [1]:
import datetime, time
from datetime import date

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

In [2]:
# index column : date
def find_previous_date(df_inv, df_his, date_current):
    i = 0
    date2 = 0
    while True:
        i += 1
        date2 = date_current - datetime.timedelta(days = i)
        if (df_inv.loc[date2]['retail'] != 0) & (df_his.loc[date2]['vol'] != '') :  
            # investors와 historical 두 개 데이터에 같이 없어야 함
            break
        if i >= 30:
            break
    return date2

In [3]:
def find_ratio(df_o, date_current):
    df_o_trans = df_o.transpose()
    df_o_trans.columns = ['before', 'after']
    df_o_trans[date_current] = (df_o_trans['after'] - df_o_trans['before']) / df_o_trans['before']
    return df_o_trans.transpose()

In [4]:
def combine_data(df_o, df_inv, df_his):
    investor_rate = pd.DataFrame()
    historical_rate = pd.DataFrame()
    date_weekday = pd.DataFrame()
    for date_current in df_o.index:
        date_previous = find_previous_date(df_inv, df_his, date_current)
        # 거래 전날 요일 구하기
        date_temp = {'date': date_current, 'weekday' : date_previous.weekday()}
        df_temp = pd.DataFrame(date_temp, index=[0]).set_index('date')
        date_weekday = pd.concat([date_weekday, df_temp], axis=0)
        date_previous_1 = find_previous_date(df_inv, df_his, date_previous)
        # 거래 전날, 전전날을 확인하고 변화정도 계산하기(find_ratio)
        df_inv_comp = df_inv.loc[[date_previous_1, date_previous]]
        df_his_comp = df_his.loc[[date_previous_1, date_previous]]
        investor_rate = pd.concat([investor_rate, find_ratio(df_inv_comp, date_current).iloc[[-1]]], axis=0)
        historical_rate = pd.concat([historical_rate, find_ratio(df_his_comp, date_current).iloc[[-1]]], axis=0)
    total = pd.concat([investor_rate, historical_rate, date_weekday, df_o ], axis=1)
    return total

In [5]:
code = {'005930': ['삼성전자', 'sec'], '005380': ['현대차', 'hyunmotor'],
                 '035420': ['NAVER', 'naver'], '033780': ['KT&G', 'ktng']}
# code = {'005930': ['삼성전자', 'sec']}

In [6]:
pkl_directory = '../data/company_daily_price_pkl'
pkl_directory = './'
pkl_directory = '../data/company_pkl/'

In [7]:
hist_column = [ 'date', 'open', 'high', 'low', 'close', 'close_cr', 'vol']
hist_column_m = [ 'date', 'open', 'high', 'low', 'close', 'vol']
inv_column = ['date', 'retail', 'foreigner', 'institution', 'financial', 'invtrust', 'pension', 'privequity',
              'bank', 'insurance', 'financeetc', 'corporateetc', 'foreigneretc']

In [8]:
# investor.pkl, historical.pkl. 읽기
for key, val in code.items():
    pkl_name= '{}.pkl'.format(val[1])
    globals()['df_{}'.format(val[1])] = pd.read_pickle('./' + pkl_name)
    globals()['df_{}'.format(val[1])]['date'] = pd.to_datetime(globals()['df_{}'.format(val[1])]['date']) #change to timestamp
    globals()['df_{}'.format(val[1])]['date'] = globals()['df_{}'.format(val[1])]['date'].dt.date  # change to datetime
    pkl_name= '{}_historical.pkl'.format(val[1])
    globals()['df_{}_{}'.format(val[1], 'historical')] = pd.read_pickle(pkl_directory + pkl_name)
    globals()['df_{}_{}'.format(val[1], 'historical')] = globals()['df_{}_{}'.format(val[1], 'historical')][hist_column_m]
    # close_cr 행을 없앰(변동이 없는 경우가 빌생하여 divided zero error 발생)
    globals()['df_{}_{}'.format(val[1], 'historical')]['date'] = globals()['df_{}_{}'.format(val[1], 'historical')]['date'].dt.date # change to datetime
    pkl_name= '{}_investors.pkl'.format(val[1])
    globals()['df_{}_{}'.format(val[1], 'investors')] = pd.read_pickle(pkl_directory + pkl_name)
    globals()['df_{}_{}'.format(val[1], 'investors')]['date'] = globals()['df_{}_{}'.format(val[1], 'investors')]['date'].dt.date # change to datetime
    
    # get combined data (eg. df_sec_combined, df_ktng_combined...)
    globals()['df_{}_{}'.format(val[1], 'combined')] = combine_data(globals()['df_{}'.format(val[1])].set_index('date'),
                            globals()['df_{}_{}'.format(val[1], 'investors')].set_index('date'),
                            globals()['df_{}_{}'.format(val[1], 'historical')].set_index('date')
                            )

In [9]:
column_selected = ['retail', 'foreigner', 'institution', 'financial', 'invtrust', 'pension', 'privequity',
              'open', 'high', 'low', 'close',  'vol', 'weekday', 'value']

In [10]:
dd = df_ktng_combined[column_selected] # company, common, df_o data combined.
dd

,retail,foreigner,institution,financial,invtrust,pension,privequity,open,high,low,close,vol,weekday,value
2022-06-20,5.957611,1.382777,0.326582,-5.192308,43.800000,0.244039,-0.226131,-0.006002,-0.003576,-0.004813,0.0,0.639612,4,2
2022-06-21,0.555802,-0.482667,-1.686069,0.642601,3.437500,-0.736189,-1.850649,0.008454,0.001196,-0.007255,-0.010791,-0.423148,0,3
2022-06-22,0.085084,-0.499857,1.037552,0.003643,-0.730382,-0.917094,-0.022901,-0.00479,-0.003584,0.00609,0.007273,-0.158932,1,2
2022-06-23,-0.655512,0.301026,-1.034130,-0.965884,-0.839552,3.865979,-0.593750,0.002407,0.003597,-0.006053,-0.01083,0.322923,2,1
2022-06-24,3.798095,-0.383436,-62.020000,54.368794,-17.558140,-0.783898,35.788462,-0.004802,-0.004779,0.001218,0.004866,0.227933,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-06,-2.487371,-0.349324,-0.808428,-0.546407,-0.537390,-0.629674,-5.015306,-0.015054,0.001074,-0.005464,0.008696,-0.240714,4,3
2023-02-07,-1.235220,-0.282157,1.394554,4.132013,-0.493661,-1.631922,-1.927573,0.010917,-0.006438,0.001099,-0.012931,-0.174638,0,2
2023-02-08,4.421123,0.718383,1.275585,-0.562272,2.456964,-4.628007,1.839726,-0.016199,-0.0054,-0.007684,-0.007642,0.164638,1,3
2023-02-09,-1.260666,-0.451439,-0.763081,-1.144466,-0.642825,-0.585129,-0.926676,-0.008782,-0.006515,-0.001106,0.006601,-0.382764,2,2


## 여기까지 완료

## 주요 지표 읽기

In [11]:
common_pkl = ["bok_rate.pkl", "cpi.pkl", "dji.pkl", "dji_future.pkl", "dxy_future.pkl", 
              "fear_greed.pkl", "fed_rate.pkl", "gold.pkl", "ixic_future.pkl", "kor_10yr_bond.pkl",
              "kor_2yr_bond.pkl", "kosdaq.pkl", "kospi.pkl", "krw_rate.pkl", "nas.pkl",
              "snp_future.pkl", "sox.pkl", "spx.pkl", "us_10yr_bond.pkl", "us_2yr_bond.pkl",
              "us_3mon_bond.pkl", "vix.pkl", "wti_future.pkl"]

In [12]:
common_pkl[0][:-4]

'bok_rate'

In [13]:
pkl_common_directory = '../data/common_pkl/'
for index_name in common_pkl:
    globals()['df_{}'.format(index_name[:-4])] = pd.read_pickle(pkl_common_directory + index_name)
    combine_data(globals()['df_{}'.format(val[1])].set_index('date'),
                            globals()['df_{}_{}'.format(val[1], 'investors')].set_index('date'),
                            globals()['df_{}_{}'.format(val[1], 'historical')].set_index('date')
                            )

In [14]:
df_kospi

,date,kospi,open,high,low,volume,kospi_cr
0,2020-01-02,2175.17,2201.21,2202.32,2171.84,494.68M,-1.02%
1,2020-01-03,2176.46,2192.58,2203.38,2165.39,631.56M,+0.06%
2,2020-01-06,2155.07,2154.97,2164.42,2149.95,592.67M,-0.98%
3,2020-01-07,2175.54,2166.60,2181.62,2164.27,568.24M,+0.95%
4,2020-01-08,2151.31,2156.27,2162.32,2137.72,913.83M,-1.11%
...,...,...,...,...,...,...,...
769,2023-02-10,2469.73,2470.65,2473.80,2454.74,662.37M,-0.48%
770,2023-02-13,2452.70,2461.96,2462.58,2440.86,486.40M,-0.69%
771,2023-02-14,2465.64,2471.33,2480.34,2459.83,349.43M,+0.53%
772,2023-02-15,2427.90,2473.09,2473.44,2424.16,387.07K,-1.53%


### feature 이름 설명
#### feature에 사용된 column name 설명

In [15]:
df_feature_name_abbr = pd.read_csv('../data/feature_name_abbr.csv')

In [16]:
standforK = lambda x : df_feature_name_abbr['Korean'][df_feature_name_abbr['abbr'] == x].values[0]
standforE = lambda x : df_feature_name_abbr['English'][df_feature_name_abbr['abbr'] == x].values[0]
standforexp = lambda x : df_feature_name_abbr['exp1'][df_feature_name_abbr['abbr'] == x].values[0]

In [17]:
# financial이 무엇인지 알기 위한 명령
c_name = 'financial'
standforK(c_name), standforE(c_name), standforexp(c_name)

('금융투자',
 'financial_Investment_amount',
 '금융투자 : 증권사, 자산운용사 등이 회사자산을 이용하여 투자할 경우')

In [18]:
c_name = 'sox'
standforK(c_name), standforE(c_name), standforexp(c_name)

('필라델피아 반도체지수', 'SOX_INDEX', nan)

### 설명에서 feature name 찾기

In [19]:
find_col_nameK = lambda x : df_feature_name_abbr[['abbr', 'Korean']][df_feature_name_abbr['Korean'].str.contains(x)]
find_col_nameE = lambda x : df_feature_name_abbr[['abbr', 'English']][df_feature_name_abbr['English'].str.contains(x, na=False, case=False)]

In [20]:
e_word = 'INDEX' # English column에서 찾기
find_col_nameE(e_word)

,abbr,English
68,dxy,Dollar_index
69,dxy_cr,Dollar_index_change_rate
84,f_g_index,FEAR_GREED_INDEX
85,sox,SOX_INDEX
86,sox_cr,SOX_INDEX_change_rate
87,cpi,Consumer Price Index
88,cpi_anticipated,anticipated consumer price index
89,cpi_previous,previous consumer price index


In [21]:
k_word = '반도체' # Korean column에서 찾기
find_col_nameK(k_word)

,abbr,Korean
85,sox,필라델피아 반도체지수
86,sox_cr,필라델피아 반도체지수 변동률
